In [1]:
from datasets import DatasetDict, Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    T5Tokenizer,
    T5ForConditionalGeneration,
    T5Config,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    GenerationConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback,
    pipeline,
)
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
import numpy as np
import time
import gc
import GPUtil
import evaluate
from numba import cuda
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
import wandb
import os
import pickle
import optuna
from typing import Dict, Union, Optional, Tuple, List, Any
import pandas as pd
import string

2023-11-15 13:37:27.969164: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 13:37:27.969219: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 13:37:27.969253: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Random seed for reproducibility
RANDOM_SEED = 42

# Default parameters for T5 model fine-tuning
PER_DEVICE_TRAIN_BATCH_SIZE = 64
PER_DEVICE_EVAL_BATCH_SIZE = 128
LEARNING_RATE = 3e-4
NUM_TRAIN_EPOCHS = 20
EARLY_STOPPING_PATIENCE = 2
NUM_BEAMS = 4

# Include BLEURT score in evaluation
INCLUDE_BLEURT = True

# Setting the DEVICE to cuda
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set path for profane word list
PROFANE_WORD_PATH = "../data/raw/en.txt"

# Set path for raw dataset dictionary
RAW_DATASET_PATH = "../data/processed/raw_dataset.pkl"
AUG_DATASET_ALL_FILTERS_PATH = "../data/processed/aug_datasets_all_filters"
AUG_DATASET_NO_TOXICITY_FILTER_PATH = "../data/processed/aug_datasets_no_toxicity_filter"
AUG_DATASET_NO_SIMILARITY_FILTER_PATH = "../data/processed/aug_datasets_no_similarity_filter"
AUG_DATASET_NO_ACCEPTABILITY_FILTER_PATH = "../data/processed/aug_datasets_no_acceptability_filter"

# Set path for txt file containing best model checkpoints
BEST_MODEL_CHECKPOINT_PATH = "../models/best_model_checkpoints.txt"

# Set path to save evaluation outputs to
VAL_PREDS_PATH = "../data/interim/val_preds.csv"
VAL_METRICS_PATH = "../data/interim/val_metrics.csv"
TEST_PREDS_PATH = "../data/final/test_preds.csv"
TEST_METRICS_PATH = "../data/final/test_metrics.csv"

# Set maximum length for input and output
MAX_INPUT_LENGTH = 64
MAX_OUTPUT_LENGTH = 64

In [3]:
# Load tokenizers and models
tokenizer_t5_small = T5Tokenizer.from_pretrained("t5-small")
model_t5_small = T5ForConditionalGeneration.from_pretrained("t5-small").to(DEVICE)
tokenizer_toxicity = RobertaTokenizer.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")
model_toxicity = RobertaForSequenceClassification.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier").to(DEVICE)
tokenizer_acceptability = AutoTokenizer.from_pretrained("iproskurina/tda-bert-en-cola")
model_acceptability = AutoModelForSequenceClassification.from_pretrained("iproskurina/tda-bert-en-cola").to(DEVICE)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassifi

In [4]:
# Load datasets
raw_datasets = DatasetDict.load_from_disk(RAW_DATASET_PATH)
aug_datasets_all_filters = DatasetDict.load_from_disk(AUG_DATASET_ALL_FILTERS_PATH)
aug_datasets_no_acceptability_filter = DatasetDict.load_from_disk(AUG_DATASET_NO_ACCEPTABILITY_FILTER_PATH)
aug_datasets_no_similarity_filter = DatasetDict.load_from_disk(AUG_DATASET_NO_SIMILARITY_FILTER_PATH)
aug_datasets_no_toxicity_filter = DatasetDict.load_from_disk(AUG_DATASET_NO_TOXICITY_FILTER_PATH)

In [120]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 10733
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 1193
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 671
    })
})

# Functions

## Debugging

In [5]:
def measure_time(func, *args, **kwargs):
    """
    Calculates the time it takes to run a function.
    """
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Function {func.__name__} took {elapsed_time:.2f} seconds to run.")
    return result

def get_gpu_memory():
    """
    Gets the GPU memory information.
    """
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    print(f"Total GPU memory: {gpu.memoryTotal}MB")
    print(f"Free GPU memory: {gpu.memoryFree}MB")
    print(f"Used GPU memory: {gpu.memoryUsed}MB")
    return gpu.memoryUsed

def force_clear_GPU_memory():
    """
    Force clears the GPU memory.
    """
    cuda.select_device(0)
    cuda.close()

def cleanup():
    """
    Cleans up the GPU memory.
    """
    gc.collect()
    torch.cuda.empty_cache()

## Baseline Models

In [6]:
# Baseline model functions
def baseline_detoxifier(text_list, profane_word_path=PROFANE_WORD_PATH):
    """
    Returns a detoxified version of the text by replacing toxic terms with blanks

    Args:
        text_list (list): list of strings to be detoxified
        toxic_list (list): list of toxic terms to be removed from text_list

    Returns:
        detoxified_text_list (list): list of detoxified strings
    """
    # Load list of profane words
    profane_words = []
    with open(profane_word_path, "r") as f:
        for line in f:
            profane_words.append(line.strip())

    # Detoxify text
    y_pred_delete = []
    for text in text_list:
        for term in profane_words:
            text = text.replace(term, "")
        y_pred_delete.append(text)

    return y_pred_delete

def bart_detoxifier(text_list):
    """
    Returns a detoxified version of the text using BART

    Args:
        text_list (list): list of strings to be detoxified

    Returns:
        detoxified_text_list (list): list of detoxified strings
    """
    pipe_bart = pipeline("text2text-generation", model="s-nlp/bart-base-detox", device=DEVICE)
    y_pred_bart = pipe_bart(text_list, max_length=MAX_OUTPUT_LENGTH, truncation=True)
    y_pred_bart = [x["generated_text"] for x in y_pred_bart]
    
    return y_pred_bart

# Helper function to add metrics to the dataframe
def add_metrics_to_df(df, model_name, metrics, save_path="../data/processed/model_metrics.csv"):
    """
    Add model metrics to a pandas dataframe
    
    Args:
    - df: pandas dataframe to add metrics to
    - model_name: name of the model
    - metrics: dictionary of evaluation metrics
    
    Returns:
    - updated pandas dataframe
    """

    # Create a df if the input df is empty
    if df is None:
        df = pd.DataFrame(columns=["Model", "BLEURT", "BLEU", "STA", "FLU", "SEM", "Overall"])

    # Check if the model name already exists in the dataframe
    if model_name in df["Model"].values:
        print(f"Model {model_name} already exists in the dataframe.")
        return df
    
    # Add the new row to the dataframe
    model_metrics_df = pd.DataFrame({
        "Model": [model_name],
        "BLEURT": [metrics["BLEURT"]],
        "BLEU": [metrics["BLEU"]],
        "STA": [metrics["STA"]],
        "FLU": [metrics["FLU"]],
        "SEM": [metrics["SEM"]],
        "Overall": [metrics["Overall"]]
    })

    # Save the dataframe to a csv file
    df = pd.concat([df, model_metrics_df], ignore_index=True)
    df.to_csv(save_path, index=False)
    
    return df


## Evaluation

In [7]:
# Initialize model variables
model_bleurt = None
model_bertscore = None
model_sacrebleu = None

def calc_sacrebleu(refs, preds):
    """
    Calculates the SacreBLEU score.

    Args:
        refs (list): List of reference sentences
        preds (list): List of predicted sentences
    
    Returns:
        results (float): SacreBLEU score
    """
    global model_sacrebleu

    if model_sacrebleu is None:
        model_sacrebleu = evaluate.load("sacrebleu")

    results = model_sacrebleu.compute(predictions=preds, references=refs)["score"]
    results = results/100

    return results

def calc_bert_score(
    refs, preds, model_type="microsoft/deberta-large-mnli", output_mean=True
    ):
    """
    Calculates BERT score per line. Note: https://docs.google.com/spreadsheets/d/1RKOVpselB98Nnh_EOC4A2BYn8_201tmPODpNWu4w7xI/edit#gid=0 lists the best performing models
    Args:
        refs (list): List of reference sentences.
        y_pred (list): List of predicted sentences.
        model_type (str): Type of BERT model to use.
        output_mean (bool): Whether to output the mean of the scores.

    Returns:
        list of precision, recall, f1 scores.

    """
    global model_bertscore

    if model_bertscore is None:
        model_bertscore = evaluate.load("bertscore")
        
    results = model_bertscore.compute(predictions=preds, references=refs, model_type=model_type)
    precision = np.array(results["precision"])
    recall = np.array(results["recall"])
    f1 = np.array(results["f1"])
    
    if output_mean:
        precision = precision.mean()
        recall = recall.mean()
        f1 = f1.mean()

    return precision, recall, f1

def calc_bleurt(refs, preds, checkpoint="BLEURT-20_D12", output_mean = True):
    """
    Calculates BLEURT score per line.

    Args:
        refs (list): List of reference sentences.
        preds (list): List of predicted sentences.
        output_type (str): Type of output to return. Either 'numpy' or 'list'.

    Returns:
        list/array of BLEURT scores.
    """
    global model_bleurt

    if model_bleurt is None:
        model_bleurt = evaluate.load("bleurt", module_type="metric", checkpoint=checkpoint)

    results = np.array(model_bleurt.compute(predictions=preds, references=refs)["scores"])

    if output_mean:
        results = results.mean()

    return results

def calc_tox_acceptability(
    data,
    tokenizer,
    model,
    output_score=True,
    output_mean=True):
    """
    Calculates toxicity and acceptability scores for a given dataset.

    Args:
        data = list of strings to be evaluated
        tokenizer = tokenizer for the model
        model = model to be used for evaluation
        output_score = whether to output the score or the label
        output_mean = whether to output the mean of the scores or the scores for each sentence
    
    Returns:
        array of toxicity and acceptability scores.
    """  
    inputs = tokenizer(data, return_tensors="pt", padding=True).to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs)["logits"]
        if output_score:
            result = torch.nn.functional.softmax(logits, dim=1)[:, 1]
        else:
            result = logits.argmax(1).data
        result = result.cpu().numpy()

    if output_mean:
        result = result.mean()
        
    return result

def evaluate_metrics(
    refs,
    preds,
    tokenizer_toxicity=tokenizer_toxicity,
    model_toxicity=model_toxicity,
    tokenizer_acceptability=tokenizer_acceptability,
    model_acceptability=model_acceptability,
    to_neutral=True,
    weights={
        "BLEU": 0.2,
        "STA": 0.4,
        "Acceptability": 0.2,
        "BERT_Score": 0.2
    },
    include_bleurt=INCLUDE_BLEURT
):
    """
    Calculates and returns a dictionary of evaluation metrics

    Args:
        refs (list): list of strings (reference)
        preds (list): list of strings (predictions)
        tokenizer_toxicity (tokenizer): tokenizer for toxicity model
        model_toxicity (model): toxicity model
        tokenizer_acceptability (tokenizer): tokenizer for acceptability model
        model_acceptability (model): acceptability model
        to_neutral (bool): whether the goal is to transfer to neutral (True) or to toxic (False)
        weights (dict): dictionary of weights for each metric
        include_bleurt (bool): whether to include BLEURT score in the output

    Returns:
        results (dict): dictionary of evaluation metrics
    """
    # Calculate BLEU score
    bleu = calc_sacrebleu(refs, preds)

    # Calculate toxicity classification
    tox_pred = calc_tox_acceptability(preds, tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)

    # Calculate style transfer accuracy as proportion of sentences that were correctly classified (as non-toxic / toxic)
    if to_neutral:
        sta_correct_label = 0
    else:
        sta_correct_label = 1

    sta_pred = (tox_pred == sta_correct_label).sum() / len(tox_pred)

    # Calculate acceptability scores
    acc_pred = calc_tox_acceptability(preds, tokenizer_acceptability, model_acceptability)

    # Calculate similarity score
    bert_score_f1 = calc_bert_score(refs, preds, model_type="distilbert-base-uncased")[2]

    # Calculate BLEURT score if include_bleurt is True
    bleurt = None
    if include_bleurt:
        bleurt = calc_bleurt(refs, preds)

    # Calculate composite score
    composite_score = weights["BLEU"] * bleu + weights["STA"] * sta_pred + weights["Acceptability"] * acc_pred + weights["BERT_Score"] * bert_score_f1

    # Return a dictionary of metrics
    results = {
        "BLEU": bleu,
        "STA": sta_pred,
        "FLU": acc_pred,
        "SEM": bert_score_f1,
        "Overall": composite_score,
    }
    if include_bleurt:
        results["BLEURT"] = bleurt
        
    return results

In [23]:
def add_preds_to_df(model_name, preds, raw_datasets=raw_datasets, use_validation=True, load_csv=True, replace_existing=True):
    """
    Add model predictions to a pandas dataframe

    Args:
    - model_name: name of the model
    - preds: list of predictions
    - test_data: whether the data is test data or validation data (True for test data, False for validation data)
    - load_csv: whether to load the existing csv file. If False, a new dataframe will be created.
    - replace_existing: whether to replace an existing column if it already exists

    Returns:
    - updated pandas dataframe
    """

    # Set save path
    if use_validation:
        save_path = VAL_PREDS_PATH
        source = raw_datasets["validation"]["source"]
        target = raw_datasets["validation"]["target"]
    else:
        save_path = TEST_PREDS_PATH
        source = raw_datasets["test"]["source"]
        target = raw_datasets["test"]["target"]

    if load_csv:
        df = pd.read_csv(save_path)
    else:
        df = pd.DataFrame({
                "source": source,
                "target": target,
            })
        
    if f"{model_name}_preds" in df.columns and not replace_existing:
        print(f"Column {model_name}_preds already exists in the dataframe.")
        return df
    
    df[f"{model_name}_preds"] = preds
    df.to_csv(save_path, index=False)

    return df

def add_metric_cols_to_preds(preds_col_name, use_validation=True):
    """
    Add metric columns to the dataframe

    Args:
    - preds_col_name: name of the column containing the predictions
    - use_validation: whether to use validation data or test data

    Returns:
    - updated dataframe
    """
    # Set save path
    if use_validation:
        save_path = VAL_PREDS_PATH
    else:
        save_path = TEST_PREDS_PATH

    # Load CSV
    df = pd.read_csv(save_path)

    # Dynamically create column names
    model_name = preds_col_name.split("_")[0]
    bleu_col_name = f"{model_name}_BLEU"
    bleurt_col_name = f"{model_name}_BLEURT"
    sta_col_name = f"{model_name}_STA"
    flu_col_name = f"{model_name}_FLU"
    sem_col_name = f"{model_name}_SEM"

    # Calculate metrics
    df[bleu_col_name] = df.apply(lambda row: calc_sacrebleu([row["target"]], [row[preds_col_name]]), axis=1)
    df[bleurt_col_name] = calc_bleurt(df["target"], df[preds_col_name], output_mean=False)
    df[sta_col_name] = 1 - calc_tox_acceptability(df[preds_col_name].tolist(), tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)
    df[flu_col_name] = calc_tox_acceptability(df[preds_col_name].tolist(), tokenizer_acceptability, model_acceptability, output_score=True, output_mean=False)
    df[sem_col_name] = calc_bert_score(df["target"], df[preds_col_name], model_type="distilbert-base-uncased", output_mean=False)[2]

    df.to_csv(save_path, index=False)

    return df

# Helper function to add metrics to the dataframe
def add_metrics_to_df(model_name, metrics, use_validation=True, load_csv=True, replace_existing=True):
    """
    Add model metrics to a pandas dataframe
    
    Args:
    - df: pandas dataframe to add metrics to
    - model_name: name of the model
    - metrics: dictionary of evaluation metrics
    - test_data: whether the data is test data or validation data (True for test data, False for validation data)
    - load_csv: whether to load the existing csv file. If False, a new dataframe will be created.
    - replace_existing: whether to replace an existing column if it already exists
    
    Returns:
    - updated pandas dataframe
    """

    # Set save path
    if use_validation:
        save_path = VAL_METRICS_PATH
    else:
        save_path = TEST_METRICS_PATH
        
    # Load the existing dataframe if it exists
    if load_csv:
        df = pd.read_csv(save_path)
    else:
        df = pd.DataFrame(columns=["Model", "BLEURT", "BLEU", "STA", "FLU", "SEM", "Overall"])

    # Check if the model name already exists in the dataframe
    if model_name in df["Model"].values and not replace_existing:
        print(f"Model {model_name} already exists in the dataframe.")
        return df

    # If replace existing, remove the existing row with the same model name
    if model_name in df["Model"].values and replace_existing:
        df = df[df["Model"] != model_name]

    # Add the new row to the dataframe
    model_metrics_df = pd.DataFrame({
        "Model": [model_name],
        "BLEURT": [metrics["BLEURT"]],
        "BLEU": [metrics["BLEU"]],
        "STA": [metrics["STA"]],
        "FLU": [metrics["FLU"]],
        "SEM": [metrics["SEM"]],
        "Overall": [metrics["Overall"]]
    })

    # Save the dataframe to a csv file
    df = pd.concat([df, model_metrics_df], ignore_index=True)
    df.to_csv(save_path, index=False)
    
    return df


In [49]:
def get_bad_words_list(dataset, tokenizer=tokenizer_toxicity, model=model_toxicity, num_layers=3, top_k=3):
    """
    Gets the top k bad words for each sentence in the dataset.

    Args:
        dataset (list): List of sentences.
        tokenizer (PreTrainedTokenizer): Tokenizer to use (toxicity classifier).
        model (PreTrainedModel): Model to use (toxicity classifier).
        num_layers (int): Number of layers to use.
        top_k (int): Number of bad words to return.

    Returns:
        bad_words_list (list): List of lists of bad words.
    """    
    bad_words_list = []

    for sentence in dataset:
        # Tokenize sentence
        inputs = tokenizer(sentence, return_tensors="pt").to(DEVICE)
        input_ids = inputs["input_ids"]

        # Get attention scores
        attention = model(input_ids, output_attentions=True)['attentions']

        # Get the last num_layers layer attention scores and average them
        attention = torch.stack(attention[-num_layers:]).mean(0)

        # Average across each head
        attention = attention.mean(1)

        # Sum each row to get the attention score for each token
        attention = attention.mean(1)

        # Exclude separator tokens and punctuation
        token_list = input_ids.squeeze().tolist()
        punctuation_ids = {tokenizer.convert_tokens_to_ids(token) for token in string.punctuation}
        exclude_ids = set([tokenizer.bos_token_id, tokenizer.eos_token_id]) | punctuation_ids

        valid_indices = [i for i, token_id in enumerate(token_list) if token_id not in exclude_ids]

        # Filter out the valid indices from the attention scores
        valid_attention = attention.squeeze()[valid_indices]

        # Get the indices of the top k tokens with the highest attention scores among valid tokens
        top_indices = valid_attention.topk(top_k).indices.tolist()
        top_token_indices = [valid_indices[i] for i in top_indices]

        # Decode the tokens
        bad_words = [tokenizer.decode(token_list[index]).strip() for index in top_token_indices]

        bad_words_list.append(bad_words)

    return bad_words_list

## Trainer Object Setup

In [9]:
def add_prefix(datasetdict, prefix="to_neutral: "):
    """Adds a prefix to the source sequence in the dataset."""
    datasetdict_copy = datasetdict.copy()
    datasetdict_copy["train"] = datasetdict_copy["train"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["validation"] = datasetdict_copy["validation"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["test"] = datasetdict_copy["test"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy = DatasetDict(datasetdict_copy)
    return datasetdict_copy

def create_bidirectional_dataset(datasets, shuffle=True):
    """
    Creates a bi-directional dataset from the original dataset.

    Args:
        datasets (DatasetDict): DatasetDict object containing the original dataset.
        shuffle (bool): Whether to shuffle the dataset or not.
    
    Returns:
        extended_datasets (DatasetDict): DatasetDict object containing the bi-directional dataset.
    """

    def bidirectional_extension(dataset):
        new_data = {
            "source": [],
            "target": []
        }
        for src, tgt in zip(dataset['source'], dataset['target']):
            new_data['source'].extend([f'to_neutral: {src}', f'to_toxic: {tgt}'])
            new_data['target'].extend([tgt, src])
        return new_data

    extended_train_data = bidirectional_extension(datasets["train"])
    extended_validation_data = bidirectional_extension(datasets["validation"])
    extended_test_data = bidirectional_extension(datasets["test"])

    extended_datasets = DatasetDict({
        "train": Dataset.from_dict(extended_train_data),
        "validation": Dataset.from_dict(extended_validation_data),
        "test": Dataset.from_dict(extended_test_data)
    })

    if shuffle:
        extended_datasets["train"] = extended_datasets["train"].shuffle(seed=RANDOM_SEED)
        
    return extended_datasets

def preprocess_dataset(dataset, tokenizer):
    """Preprocesses a dataset using a tokenizer."""
    def preprocess_function(examples, tokenizer):
        """Preprocess function for T5."""
        model_inputs = tokenizer(
            examples["source"],
            text_target=examples["target"],
            max_length=MAX_INPUT_LENGTH,
            truncation=True,
        )
        return model_inputs

    return dataset.map(
        preprocess_function,
        fn_kwargs={'tokenizer': tokenizer},
        batched=True,
        remove_columns=["source", "target"],
    )

def post_process(preds, refs, tokenizer):
    """
    Post-process function for T5.

    Args:
        preds (list): list of predicted sequences
        refs (list): list of reference sequences
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding

    Returns:
        decoded_preds (list): list of decoded predicted sequences
        decoded_refs (list): list of decoded reference sequences
    """
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    refs = np.where(refs != -100, refs, tokenizer.pad_token_id)
    decoded_refs = tokenizer.batch_decode(refs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_refs = [ref.strip() for ref in decoded_refs]

    return decoded_preds, decoded_refs

def post_process_preds(preds, tokenizer):
    """
    Post-process function for T5 (only for predictions)

    Args:
        preds (list): list of predicted sequences
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding

    Returns:
        decoded_preds (list): list of decoded predicted sequences
    """
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]

    return decoded_preds

def compute_metrics(eval_preds, tokenizer):
    """
    Function to calculate the metrics for trainer.evaluate().

    Args:
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        eval_preds (tuple): Tuple containing the predictions and references

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # Post-process the predictions and references
    decoded_preds, decoded_refs = post_process(preds, refs, tokenizer)
    
    # Evaluate metrics
    return evaluate_metrics(
        decoded_refs,
        decoded_preds,
        tokenizer_toxicity=tokenizer_toxicity,
        model_toxicity=model_toxicity,
        tokenizer_acceptability=tokenizer_acceptability,
        model_acceptability=model_acceptability,
        include_bleurt=INCLUDE_BLEURT
    )

def compute_metrics_bd(eval_preds, tokenizer, bd_dataset, shuffled_data=False):
    """
    Function to calculate the metrics for trainer.evaluate().
    This function is for the bi-directional model.
    
    Args:
        eval_preds (tuple): Tuple containing the predictions and references
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        shuffled_data (bool): Whether the data is shuffled or not
        bd_dataset (DatasetDict): Bidirectional dataset to use for testing created using create_bidirectional_datasets
                                  For example, raw_datasets_bd["validation"] or raw_datasets_bd["test"]

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # Post-process the predictions and references
    decoded_preds, decoded_refs = post_process(preds, refs, tokenizer)
    
    # If shuffled data is false, have to_neutral_preds and to_neutral_refs just be predictions and refs with even indices
    if not shuffled_data:
        to_neutral_preds = decoded_preds[::2]
        to_neutral_refs = decoded_refs[::2]
    # Otherwise, get the indices to use when splitting predictions and refs to to_neutral and to_toxic
    else:
        # Get the indices to use when splitting predictions and refs to to_neutral and to_toxic
        to_neutral_idx = [i for i, input_sentence in enumerate(bd_dataset['source']) if input_sentence.startswith("to_neutral")]

        # Retrieve based on the indices
        to_neutral_preds = [decoded_preds[i] for i in to_neutral_idx]
        to_neutral_refs = [decoded_refs[i] for i in to_neutral_idx]
    
    # Evaluate metrics for to_neutral
    to_neutral_metrics = evaluate_metrics(
        to_neutral_refs,
        to_neutral_preds,
        include_bleurt=INCLUDE_BLEURT
    )

    # Return dictionary of to_neutral metrics
    return to_neutral_metrics

def setup_trainer(output_dir_name,
                train_dataset,
                eval_dataset,
                compute_metrics,
                model_checkpoint="t5-small",
                per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                num_train_epochs=NUM_TRAIN_EPOCHS,
                max_length=MAX_OUTPUT_LENGTH,
                num_beams=NUM_BEAMS,
                early_stopping_patience=EARLY_STOPPING_PATIENCE,
                report_to="wandb",
                ):
    """
    Set up a Seq2SeqTrainer object for training a T5 model.

    Default parameters based on this: https://github.com/google-research/text-to-text-transfer-transformer/blob/main/t5/models/hf_model.py#L55

    Args:
        output_dir_name (str): What to name the model in the output directory.
        train_dataset (Dataset): Training dataset.
        eval_dataset (Dataset): Evaluation dataset.
        compute_metrics (function): Function to compute metrics. Change this to compute_metrics_bd if using a bi-directional model.
        model_checkpoint (str): Model checkpoint to use.
        per_device_train_batch_size (int): Batch size for training.
        per_device_eval_batch_size (int): Batch size for evaluation.
        learning_rate (float): Learning rate.
        num_train_epochs (int): Number of training epochs.
        max_length (int): Maximum length of the output sequence.
        num_beams (int): Number of beams for beam search.
        early_stopping_patience (int): Number of epochs to wait before early stopping.
        report_to (str): Where to report results to. Either "wandb" or "none".

    Returns:
        Seq2SeqTrainer: Trainer object for training the T5 model.
    """
    
    # Instantiate model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    # Define the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model, return_tensors="pt", padding=True)

    # Define generation config
    generation_config = GenerationConfig(
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        pad_token_id=model.config.pad_token_id,
        decoder_start_token_id=model.config.pad_token_id
        )

    # Save the generation config
    gen_config_path = f"../models/{output_dir_name}/generation_config"
    generation_config.save_pretrained(gen_config_path)

    # Define the training arguments
    args = Seq2SeqTrainingArguments(
        output_dir=f'../models/{output_dir_name}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        learning_rate=learning_rate, 
        predict_with_generate=True,
        generation_config=gen_config_path,
        fp16=True,
        report_to=report_to,
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="Overall",
        greater_is_better=True,
        generation_max_length=max_length,
    )
   
    # Instantiate the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=partial(compute_metrics, tokenizer=tokenizer),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]

    )

    return trainer

def setup_trainer(output_dir_name,
                train_dataset,
                eval_dataset,
                compute_metrics,
                model_checkpoint="t5-small",
                per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                num_train_epochs=NUM_TRAIN_EPOCHS,
                max_length=MAX_OUTPUT_LENGTH,
                num_beams=NUM_BEAMS,
                early_stopping_patience=EARLY_STOPPING_PATIENCE,
                report_to="wandb",
                ):
    """
    Set up a Seq2SeqTrainer object for training a T5 model.

    Default parameters based on this: https://github.com/google-research/text-to-text-transfer-transformer/blob/main/t5/models/hf_model.py#L55

    Args:
        output_dir_name (str): What to name the model in the output directory.
        train_dataset (Dataset): Training dataset.
        eval_dataset (Dataset): Evaluation dataset.
        compute_metrics (function): Function to compute metrics. Change this to compute_metrics_bd if using a bi-directional model.
        model_checkpoint (str): Model checkpoint to use.
        per_device_train_batch_size (int): Batch size for training.
        per_device_eval_batch_size (int): Batch size for evaluation.
        learning_rate (float): Learning rate.
        num_train_epochs (int): Number of training epochs.
        max_length (int): Maximum length of the output sequence.
        num_beams (int): Number of beams for beam search.
        early_stopping_patience (int): Number of epochs to wait before early stopping.
        report_to (str): Where to report results to. Either "wandb" or "none".

    Returns:
        Seq2SeqTrainer: Trainer object for training the T5 model.
    """
    
    # Instantiate model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    # Define the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model, return_tensors="pt", padding=True)

    # Define generation config
    generation_config = GenerationConfig(
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        pad_token_id=model.config.pad_token_id,
        decoder_start_token_id=model.config.pad_token_id
        )

    # Save the generation config
    gen_config_path = f"../models/{output_dir_name}/generation_config"
    generation_config.save_pretrained(gen_config_path)

    # Define the training arguments
    args = Seq2SeqTrainingArguments(
        output_dir=f'../models/{output_dir_name}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        learning_rate=learning_rate, 
        predict_with_generate=True,
        generation_config=gen_config_path,
        fp16=True,
        report_to=report_to,
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="Overall",
        greater_is_better=True,
        generation_max_length=max_length,
    )
   
    # Instantiate the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=partial(compute_metrics, tokenizer=tokenizer),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
    )

    return trainer

def training_pipeline(model_name, project_name="t5-detox", model_checkpoint="t5-small", use_validation=True, raw_datasets=raw_datasets, bidirectional=False, shuffle=False, do_train=True):
    """
    Pipeline for training a T5 model. Saves the best model checkpoint to a txt file. Can also be used for evaluating a model (use test set instead of validation set).

    Args:
        model_name (str): Name of the model to name the output directory and wandb run.
        project_name (str): Name of the wandb project.
        model_checkpoint (str): Model checkpoint to use.
        use_validation (bool): Whether to use the validation set or not.
        raw_datasets (DatasetDict): DatasetDict object containing the original dataset.
        bidirectional (bool): Whether to use a bi-directional model or not.
        shuffle (bool): Whether to shuffle the dataset or not.
        do_train (bool): Whether to train the model or not.

    Returns:
        trainer (Seq2SeqTrainer): Trainer object for training the T5 model.
    """
    
    # Preprocess dataset (add prefixes / make bidirectional)
    if bidirectional:
        raw_datasets = create_bidirectional_dataset(raw_datasets, shuffle=shuffle)
    else:
        raw_datasets = add_prefix(raw_datasets)

    # Tokenize dataset
    tokenized_datasets = preprocess_dataset(raw_datasets, tokenizer_t5_small)

    # Define compute_metrics function depending on bidirectional or not
    if bidirectional and use_validation:
        bd_dataset = raw_datasets["validation"]
    elif bidirectional and not use_validation:
        bd_dataset = raw_datasets["test"]
    else:
        bd_dataset = None

    compute_metrics_fn = partial(compute_metrics_bd, bd_dataset=bd_dataset, shuffled_data=shuffle) if bd_dataset else compute_metrics

    # Setup trainer
    trainer = setup_trainer(
        output_dir_name=model_name,
        model_checkpoint=model_checkpoint,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"] if use_validation else tokenized_datasets["test"],
        compute_metrics=compute_metrics_fn
    )

    if do_train:
        # Initialize wandb
        wandb.init(project=project_name, name=model_name)
        trainer.train()
        wandb.finish()

        # Get the best checkpoint path for the model
        checkpoint_path = trainer.state.best_model_checkpoint

        # Save the checkpoint path for the best model
        with open(BEST_MODEL_CHECKPOINT_PATH, "a") as file:
            file.write(f"{model_name}: {checkpoint_path}\n")

    return trainer, tokenized_datasets

## T5 Evaluation Helper Functions

In [110]:
def get_model_checkpoints():
    # Get checkpoint values for the best models
    with open(BEST_MODEL_CHECKPOINT_PATH, "r") as f:
        best_model_checkpoints = f.readlines()

    # Convert to a dictionary
    best_model_checkpoints_dict = {}
    for line in best_model_checkpoints:
        model_name, checkpoint_path = line.split(": ")
        best_model_checkpoints_dict[model_name] = checkpoint_path.strip()

    return best_model_checkpoints_dict

model_checkpoints = get_model_checkpoints()

def get_t5_preds_metrics(model_checkpoint,
                         raw_datasets=raw_datasets,
                         bidirectional=False,
                         shuffle=False,
                         use_validation=True,
                         do_train=False,
                         tokenizer=tokenizer_t5_small
                         ):
    """
    Returns the predictions and metrics for a fine-tuned T5 model.
    """
    
    # Setup training pipeline
    trainer, trainer_tokenized_ds = training_pipeline(
        model_name="n/a",
        project_name="n/a",
        model_checkpoint=model_checkpoint,
        use_validation=use_validation,
        raw_datasets=raw_datasets,
        bidirectional=bidirectional,
        shuffle=shuffle,
        do_train=do_train
    )

    # Get raw predictions
    if use_validation:
        trainer_preds_raw = trainer.predict(trainer_tokenized_ds["validation"])
    else:
        trainer_preds_raw = trainer.predict(trainer_tokenized_ds["test"])

    # Get encoded predictions and metrics
    trainer_preds_encoded, trainer_metrics = trainer_preds_raw.predictions, trainer_preds_raw.metrics

    # Post-process predictions
    if isinstance(trainer_preds_encoded, tuple):
        trainer_preds_encoded = trainer_preds_encoded[0]

    trainer_preds_decoded = tokenizer.batch_decode(trainer_preds_encoded, skip_special_tokens=True)
    trainer_preds_decoded = [pred.strip() for pred in trainer_preds_decoded]

    #Return trainer metrics in the same format as evaluate_metrics
    trainer_metrics = {
        "BLEU": trainer_metrics["test_BLEU"],
        "BLEURT": trainer_metrics["test_BLEURT"],
        "STA": trainer_metrics["test_STA"],
        "FLU": trainer_metrics["test_FLU"],
        "SEM": trainer_metrics["test_SEM"],
        "Overall": trainer_metrics["test_Overall"]
    }
        
    # Return predictions and metrics
    return trainer_preds_decoded, trainer_metrics
    
def compare_outputs(
    df,
    model_to_compare,
    bad_words_col_1 = "source",
    bad_words_col_2 = "T5-Small-UD_preds",
    num_examples = 5,
    num_layers = 5,
    top_k = 3,
    show_bart = True,
    print_toxic_words = True,
    random_state = RANDOM_SEED,
):
    """
    Prints the source, target, and outputs of two models for comparison.
    """
    # Randomly sample the filtered dataset
    df_sample = df.sample(n=num_examples, random_state=random_state)

    # Get toxic words for the source and target
    if print_toxic_words:
        bad_words_1 = get_bad_words_list(df_sample[bad_words_col_1], num_layers=num_layers, top_k=top_k)
        bad_words_2 = get_bad_words_list(df_sample[bad_words_col_2], num_layers=num_layers, top_k=top_k)
    else:
        bad_words_1 = list(range(num_examples))
        bad_words_2 = list(range(num_examples))

    # Strip _preds from bad_words_col_1 and bad_words_col_2
    bad_words_col_1 = bad_words_col_1.replace("_preds", "")
    bad_words_col_2 = bad_words_col_2.replace("_preds", "")
    
    # Print the lines
    if model_to_compare is None:
        for source, target, bart, t5_small_ud, bad_words_1, bad_words_2, in zip(df_sample["source"], df_sample["target"], df_sample["BART_preds"], df_sample["T5-Small-UD_preds"], bad_words_1, bad_words_2):
            print(f"Source: {source}")
            print(f"Target: {target}")
            if show_bart:
                print(f"BART: {bart}")
            print(f"T5-Small-UD: {t5_small_ud}")
            if print_toxic_words:
                print(f"'Toxic' words in {bad_words_col_1}: {bad_words_1}")
                print(f"'Toxic' words in {bad_words_col_2}: {bad_words_2}")
            print()
    else:
        for source, target, bart, t5_small_ud, model_to_compare_out, bad_words_1, bad_words_2, in zip(df_sample["source"], df_sample["target"], df_sample["BART_preds"], df_sample["T5-Small-UD_preds"], df_sample[model_to_compare + "_preds"], bad_words_1, bad_words_2):
            print(f"Source: {source}")
            print(f"Target: {target}")
            if show_bart:
                print(f"BART: {bart}")
            print(f"T5-Small-UD: {t5_small_ud}")
            print(f"{model_to_compare}: {model_to_compare_out}")
            if print_toxic_words:
                print(f"'Toxic' words in {bad_words_col_1}: {bad_words_1}")
                print(f"'Toxic' words in {bad_words_col_2}: {bad_words_2}")
            print()

# Evaluation Using Validation Set

## Baseline: DELETE

In [10]:
delete_val_preds = baseline_detoxifier(raw_datasets["validation"]["source"])
df_val_preds = add_preds_to_df("DELETE", delete_val_preds, use_validation=True, load_csv=False)

delete_val_metrics = evaluate_metrics(raw_datasets["validation"]["target"], delete_val_preds)
df_val_metrics = add_metrics_to_df("DELETE", delete_val_metrics, use_validation=True, load_csv=False)

/tmp/ipykernel_6611/1231404667.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, model_metrics_df], ignore_index=True)


## Baseline: BART

In [11]:
bart_val_preds = bart_detoxifier(raw_datasets["validation"]["source"])
df_val_preds = add_preds_to_df("BART", bart_val_preds, use_validation=True, load_csv=True)

bart_val_metrics = evaluate_metrics(raw_datasets["validation"]["target"], bart_val_preds)
df_val_metrics = add_metrics_to_df("BART", bart_val_metrics, use_validation=True, load_csv=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## T5-Small (Unidirectional)

In [12]:
# List keys in the dictionary
model_checkpoints.keys()

dict_keys(['t5_small_unidir', 't5_small_bidir_noshuf', 't5_small_bidir_shuf', 't5_small_aug_all', 't5_small_aug_noaccept', 't5_small_aug_nosim', 't5_small_aug_notox'])

In [15]:
# Get predictions and metrics
t5_small_ud_val_preds, t5_small_ud_val_metrics = get_t5_preds_metrics(
    model_checkpoint=model_checkpoints["t5_small_unidir"],
    use_validation=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
# Add predictions to dataframe and per-row metrics
df_val_preds = add_preds_to_df("T5-Small-UD", t5_small_ud_val_preds, use_validation=True, load_csv=True)
df_val_preds = add_metric_cols_to_preds("BART_preds", use_validation=True)
df_val_preds = add_metric_cols_to_preds("T5-Small-UD_preds", use_validation=True)

# Add aggregated metrics to dataframe
df_val_metrics = add_metrics_to_df("T5-Small-UD", t5_small_ud_val_metrics, use_validation=True, load_csv=True)

Assess examples and compare to baseline

In [55]:
# Load csv file
df_val_preds = pd.read_csv(VAL_PREDS_PATH)
df_val_preds_metrics = add_metric_cols_to_preds(df_val_preds, "BART_preds")
df_val_preds_metrics = add_metric_cols_to_preds(df_val_preds_metrics, "T5-Small-UD_preds")
df_val_preds_metrics.head()

,source,target,DELETE_preds,BART_preds,T5-Small-UD_preds,BART_BLEU,BART_BLEURT,BART_STA,BART_FLU,BART_SEM,T5-Small-UD_BLEU,T5-Small-UD_BLEURT,T5-Small-UD_STA,T5-Small-UD_FLU,T5-Small-UD_SEM
0,Rt my bitch got fans give em an autograph smh ...,Rt my girl got fans give em an autograph smh i...,Rt my got fans give em an autograph smh i lov...,Rt my girl got fans give em an autograph smh i...,I got fans give em an autograph tho,1.000000,1.039609,1,0.554691,1.000000,0.334014,-0.373949,1,0.768761,0.901671
1,""" redditor for 26 minutes "" fuck off",Redditor for 26 minutes,""" redditor for 26 minutes "" off",redditor for 26 minutes,Redditor for 26 minutes,0.594604,1.110395,1,0.115584,1.000000,1.000000,1.110395,1,0.170830,1.000000
2,"Yup then i'll unfollow you, just to be a bitch",Yup then i'll unfollow you.,"Yup then i'll unfollow you, just to be a","Yup then i'll unfollow you, just to be rude",Yup then i'll unfollow you,0.392815,-0.292918,1,0.981803,0.928382,0.818731,0.937014,1,0.967795,0.976986
3,And don't say he can't fucking do it.,Don't say he can't do it.,And don't say he can't ing do it.,And don't say he can't do it.,And don't say he can't do it.,0.680375,0.687025,1,0.970324,0.979388,0.680375,0.687025,1,0.970324,0.979388
4,In the words of george carlin the planet is do...,In the words of George Carlin “the planet is d...,In the words of george carlin the planet is do...,In the words of George Carlin the planet is do...,In the words of george carlin the planet is do...,0.603944,0.700623,1,0.957270,0.970476,0.488101,0.700623,1,0.245888,0.970476


In [26]:
df_val_metrics

,Model,BLEURT,BLEU,STA,FLU,SEM,Overall
0,DELETE,-0.227430,0.529101,0.659681,0.478651,0.911821,0.647787
1,BART,0.466564,0.701595,0.917854,0.718025,0.945139,0.840093
2,T5-Small-BD,0.217048,0.611682,0.912825,0.711991,0.926827,0.815230
3,T5-Small-UD,0.206119,0.606150,0.902766,0.707668,0.925923,0.809055


Based on this example, the largest differences between BART and T5-Small-UD are in relation to BLEURT.

Comparison of BLEURT

In [79]:
# Filter the dataset to rows where BART has
df_BLEURT_BART_vs_T5 = df_val_preds[(df_val_preds["BART_BLEURT"] >= df_val_preds["T5-Small-UD_BLEURT"] * 1.2)]

# Compmare outputs
compare_outputs(
    df=df_BLEURT_BART_vs_T5,
    model_to_compare=None,
    print_toxic_words=False
)

Source: Who the fuck cuts out part of the state of the union?
Target: Who cuts out part of the state of tge union.
BART: Who cuts out part of the state of the union?
T5-Small-UD: Who cuts out part of the state of the union?

Source: Nah yu seen how i act n public sober n shit
Target: Nah you’ve seen how i act in public, sober and all.
BART: Nah yu seen how i act in public sober
T5-Small-UD: Nah you have seen how i act n public sober

Source: Shit chillen fresh out the shower
Target: Chillen fresh out of the shower
BART: Chillen fresh out of the shower
T5-Small-UD: Coolen fresh out the shower

Source: These repubicans are like cannibalistic pirañas.
Target: These republicans are taking advantage.
BART: These republicans are not good.
T5-Small-UD: These repubicans are like cannibalistic people.

Source: I mean very irritating to see females fightin over some dick
Target: Its irritating to see females fighting over boys
BART: It is very irritating to see females fighting over boys
T5-Smal

Comparison of STA

In [80]:
# Filter the dataset to rows where BART is better than T5-Small-UD in terms of STA
df_STA_BART_vs_T5 = df_val_preds[(df_val_preds["BART_STA"] == 1) & (df_val_preds["T5-Small-UD_STA"] == 0)]

# Compare outputs
compare_outputs(
    df=df_BLEURT_BART_vs_T5,
    model_to_compare=None,
    print_toxic_words=True
)

Source: Who the fuck cuts out part of the state of the union?
Target: Who cuts out part of the state of tge union.
BART: Who cuts out part of the state of the union?
T5-Small-UD: Who cuts out part of the state of the union?
'Toxic' words in source: ['fuck', 'the', 'part']
'Toxic' words in T5-Small-UD_preds: ['state', 'cuts', 'part']

Source: Nah yu seen how i act n public sober n shit
Target: Nah you’ve seen how i act in public, sober and all.
BART: Nah yu seen how i act in public sober
T5-Small-UD: Nah you have seen how i act n public sober
'Toxic' words in source: ['shit', 'i', 'u']
'Toxic' words in T5-Small-UD_preds: ['i', 'sober', 'n']

Source: Shit chillen fresh out the shower
Target: Chillen fresh out of the shower
BART: Chillen fresh out of the shower
T5-Small-UD: Coolen fresh out the shower
'Toxic' words in source: ['it', 'Sh', 'chill']
'Toxic' words in T5-Small-UD_preds: ['Cool', 'en', 'shower']

Source: These repubicans are like cannibalistic pirañas.
Target: These republican

## T5-Small (Bidirectional)

In [27]:
model_checkpoints

{'t5_small_unidir': '../models/t5_small_unidir/checkpoint-840',
 't5_small_bidir_noshuf': '../models/t5_small_bidir_noshuf/checkpoint-2352',
 't5_small_bidir_shuf': '../models/t5_small_bidir_shuf/checkpoint-3024',
 't5_small_aug_all': '../models/t5_small_aug_all/checkpoint-2592',
 't5_small_aug_noaccept': '../models/t5_small_aug_noaccept/checkpoint-1620',
 't5_small_aug_nosim': '../models/t5_small_aug_nosim/checkpoint-2592',
 't5_small_aug_notox': '../models/t5_small_aug_notox/checkpoint-1944'}

In [28]:
# Get predictions and metrics
t5_small_bd_val_preds, t5_small_bd_val_metrics = get_t5_preds_metrics(
    model_checkpoint=model_checkpoints["t5_small_bidir_shuf"],
    use_validation=True)

# Add predictions to dataframe
df_val_preds = add_preds_to_df("T5-Small-BD", t5_small_bd_val_preds, use_validation=True, load_csv=True)

# Add metrics to dataframe
df_val_metrics = add_metrics_to_df("T5-Small-BD", t5_small_bd_val_metrics, use_validation=True, load_csv=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
# Add per-row metrics to dataframe
df_val_preds = add_metric_cols_to_preds("T5-Small-BD_preds", use_validation=True)

In [32]:
df_val_metrics

,Model,BLEURT,BLEU,STA,FLU,SEM,Overall
0,DELETE,-0.227430,0.529101,0.659681,0.478651,0.911821,0.647787
1,BART,0.466564,0.701595,0.917854,0.718025,0.945139,0.840093
2,T5-Small-UD,0.206119,0.606150,0.902766,0.707668,0.925923,0.809055
3,T5-Small-BD,0.217048,0.611682,0.912825,0.711991,0.926827,0.815230


Comparison of BLEURT vs. T5-Small-UD

In [109]:
# Filter the dataset to rows where BLEURT of T5-Small-BD exceeds T5-Small-UD by 20% and are not identical
df_BLEURT_T5_BD_vs_T5_UD = df_val_preds[(df_val_preds["T5-Small-BD_BLEURT"] >= df_val_preds["T5-Small-UD_BLEURT"] * 1.2) & (df_val_preds["T5-Small-BD_BLEURT"] != df_val_preds["T5-Small-UD_BLEURT"])]

# Compare outputs
compare_outputs(
    df=df_BLEURT_T5_BD_vs_T5_UD,
    model_to_compare="T5-Small-BD",
    num_examples=5,
    num_layers=5,
    top_k=2,
    print_toxic_words=False,
    show_bart=False
)


Source: He dont care what happens to blacks.
Target: He dont care what happens to these persons
T5-Small-UD: He dont care what happens to blacks.
T5-Small-BD: He doesn't care what happens to blacks.

Source: Has written some good stuff in the past but this is just dumb.
Target: Has written some good stuff in the past but this is not good.
T5-Small-UD: Has written some good stuff in the past but this is just bad.
T5-Small-BD: Has written some good stuff in the past but this is just not good.

Source: I'm just glad my phone doesn't autocorrect my " fucks " to " ducks "
Target: I'm just glad my phone doesn't autocorrect mean words
T5-Small-UD: I'm just glad my phone doesn't autocorrect my " to " ducks "
T5-Small-BD: I'm just glad my phone doesn't autocorrect my to ducks

Source: Wanna play with me bitch!!!!! * kevin hart voice *
Target: Do you want to play with me! *Kevin Hart voice*
T5-Small-UD: Wanna play with me!!!!! *kevin hart voice *
T5-Small-BD: Wanna play with me!!!!! Kevin hart v

Comparison of STA vs. T5-Small-UD

In [111]:
# Filter the dataset to rows where BART is better than T5-Small-UD in terms of STA
df_STA_T5_BD_vs_T5_UD = df_val_preds[(df_val_preds["T5-Small-BD_STA"] == 1) & (df_val_preds["T5-Small-UD_STA"] == 0)]

# Compare outputs
compare_outputs(
    df=df_STA_T5_BD_vs_T5_UD,
    model_to_compare="T5-Small-BD",
    num_examples=5,
    num_layers=5,
    top_k=2,
    print_toxic_words=True,
    show_bart=False
)


Source: You already burned down to a crisp in fort mac by your own stupidity, already.
Target: You already lost in fort mac.
T5-Small-UD: You already burned down to a crisp in fort mac by your own ignorance, already.
T5-Small-BD: You already burned down to a crisp in fort mac by yourself, already.
'Toxic' words in source: ['stupidity', 'burned']
'Toxic' words in T5-Small-UD: ['You', 'burned']

Source: True conservatives, that are not white racists, will have to find a new home.
Target: Correct
T5-Small-UD: True conservatives, that are not white, will have to find a new home.
T5-Small-BD: True conservatives will have to find a new home.
'Toxic' words in source: ['white', 'racists']
'Toxic' words in T5-Small-UD: ['white', 'True']

Source: So you would love to see the commys win over a duly elected democratic president, man you are sick
Target: So you would love to see the communist win over a duly elected democratic president
T5-Small-UD: So you would love to see the commys win over a du

## T5-Small (Unidirectional, Data Augmentation)

In [112]:
model_checkpoints

{'t5_small_unidir': '../models/t5_small_unidir/checkpoint-840',
 't5_small_bidir_noshuf': '../models/t5_small_bidir_noshuf/checkpoint-2352',
 't5_small_bidir_shuf': '../models/t5_small_bidir_shuf/checkpoint-3024',
 't5_small_aug_all': '../models/t5_small_aug_all/checkpoint-2592',
 't5_small_aug_noaccept': '../models/t5_small_aug_noaccept/checkpoint-1620',
 't5_small_aug_nosim': '../models/t5_small_aug_nosim/checkpoint-2592',
 't5_small_aug_notox': '../models/t5_small_aug_notox/checkpoint-1944'}

In [113]:
# Get predictions and metrics
t5_small_aug_all_val_preds, t5_small_aug_all_val_metrics = get_t5_preds_metrics(
    model_checkpoint=model_checkpoints["t5_small_aug_all"],
    use_validation=True)

# Add predictions to dataframe
df_val_preds = add_preds_to_df("T5-Small-Aug-All", t5_small_aug_all_val_preds, use_validation=True, load_csv=True)

# Add metrics to dataframe
df_val_metrics = add_metrics_to_df("T5-Small-Aug-All", t5_small_aug_all_val_metrics, use_validation=True, load_csv=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [115]:
# Add row metrics to dataframe
df_val_preds = add_metric_cols_to_preds("T5-Small-Aug-All_preds", use_validation=True)

In [116]:
# Get predictions and metrics
t5_small_aug_notox_val_preds, t5_small_aug_notox_val_metrics = get_t5_preds_metrics(
    model_checkpoint=model_checkpoints["t5_small_aug_notox"],
    use_validation=True)

# Add predictions to dataframe
df_val_preds = add_preds_to_df("T5-Small-Aug-NoTox", t5_small_aug_notox_val_preds, use_validation=True, load_csv=True)

# Add metrics to dataframe
df_val_metrics = add_metrics_to_df("T5-Small-Aug-NoTox", t5_small_aug_notox_val_metrics, use_validation=True, load_csv=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [117]:
# Get predictions and metrics
t5_small_aug_nosim_val_preds, t5_small_aug_nosim_val_metrics = get_t5_preds_metrics(
    model_checkpoint=model_checkpoints["t5_small_aug_nosim"],
    use_validation=True)

# Add predictions to dataframe
df_val_preds = add_preds_to_df("T5-Small-Aug-NoSim", t5_small_aug_nosim_val_preds, use_validation=True, load_csv=True)

# Add metrics to dataframe
df_val_metrics = add_metrics_to_df("T5-Small-Aug-NoSim", t5_small_aug_nosim_val_metrics, use_validation=True, load_csv=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [118]:
# Get predictions and metrics
t5_small_aug_noflu_val_preds, t5_small_aug_noflu_val_metrics = get_t5_preds_metrics(
    model_checkpoint=model_checkpoints["t5_small_aug_noaccept"],
    use_validation=True)

# Add predictions to dataframe
df_val_preds = add_preds_to_df("T5-Small-Aug-NoFlu", t5_small_aug_noflu_val_preds, use_validation=True, load_csv=True)

# Add metrics to dataframe
df_val_metrics = add_metrics_to_df("T5-Small-Aug-NoFlu", t5_small_aug_noflu_val_metrics, use_validation=True, load_csv=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [119]:
df_val_metrics

,Model,BLEURT,BLEU,STA,FLU,SEM,Overall
0,DELETE,-0.227430,0.529101,0.659681,0.478651,0.911821,0.647787
1,BART,0.466564,0.701595,0.917854,0.718025,0.945139,0.840093
2,T5-Small-UD,0.206119,0.606150,0.902766,0.707668,0.925923,0.809055
3,T5-Small-BD,0.217048,0.611682,0.912825,0.711991,0.926827,0.815230
4,T5-Small-Aug-All,0.204206,0.593218,0.916178,0.714662,0.925547,0.813157
5,T5-Small-Aug-NoTox,0.208252,0.598625,0.892707,0.713698,0.925555,0.804659
6,T5-Small-Aug-NoSim,0.211811,0.591599,0.917854,0.719671,0.924972,0.814390
7,T5-Small-Aug-NoFlu,0.192281,0.595025,0.903604,0.709992,0.925311,0.807508


In [ ]:
# Multiply each by 1

# T5 Model With Negative Lexically Constrained Decoding

The first step is to generate a list of bad words per sentence in the test dataset

In [71]:
tokenizer_toxicity = RobertaTokenizer.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")
model_toxicity = RobertaForSequenceClassification.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [106]:
tokenizer_toxicity = RobertaTokenizer.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")
model_toxicity = RobertaForSequenceClassification.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")

def get_bad_words_list(dataset, tokenizer, model, num_layers=3, top_k=3):
    """
    Gets the top k bad words for each sentence in the dataset.

    Args:
        dataset (list): List of sentences.
        tokenizer (PreTrainedTokenizer): Tokenizer to use (toxicity classifier).
        model (PreTrainedModel): Model to use (toxicity classifier).
        num_layers (int): Number of layers to use.
        top_k (int): Number of bad words to return.

    Returns:
        bad_words_list (list): List of lists of bad words.
    """    
    bad_words_list = []

    for sentence in dataset:
        # Tokenize sentence
        inputs = tokenizer(sentence, return_tensors="pt").to(DEVICE)
        input_ids = inputs["input_ids"]

        # Get attention scores
        attention = model(input_ids, output_attentions=True)['attentions']

        # Get the last num_layers layer attention scores and average them
        attention = torch.stack(attention[-num_layers:]).mean(0)

        # Average across each head
        attention = attention.mean(1)

        # Sum each row to get the attention score for each token
        attention = attention.mean(1)

        # Exclude separator tokens and punctuation
        token_list = input_ids.squeeze().tolist()
        punctuation_ids = {tokenizer.convert_tokens_to_ids(token) for token in string.punctuation}
        exclude_ids = set([tokenizer.bos_token_id, tokenizer.eos_token_id]) | punctuation_ids

        valid_indices = [i for i, token_id in enumerate(token_list) if token_id not in exclude_ids]

        # Filter out the valid indices from the attention scores
        valid_attention = attention.squeeze()[valid_indices]

        # Get the indices of the top k tokens with the highest attention scores among valid tokens
        top_indices = valid_attention.topk(top_k).indices.tolist()
        top_token_indices = [valid_indices[i] for i in top_indices]

        # Decode the tokens
        bad_words = [tokenizer.decode(token_list[index]).strip() for index in top_token_indices]

        bad_words_list.append(bad_words)

    return bad_words_list

In [107]:
bad_words_list = get_bad_words_list(raw_datasets["test"]["source"], tokenizer_toxicity, model_toxicity)